In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!pip install cleverhans==3.0.1 tensorflow==1.8.0

     |████████████████████████████████| 204kB 3.4MB/s 
     |████████████████████████████████| 49.1MB 1.3MB/s 
     |████████████████████████████████| 163kB 28.1MB/s 
     |████████████████████████████████| 51kB 22.8MB/s 
     |████████████████████████████████| 3.1MB 30.6MB/s 
     |████████████████████████████████| 890kB 36.2MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107221 sha256=87fc249ef4e30706b814122834ddb0d31b1dd39bb87b7d6c84a3f60d84207506
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
ERROR: magenta 0.3.19 has requirement tensorflow>=1.12.0, but you'll have tensorflow 1.8.0 which is incompatible.
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.1.0
    Uninstalling bleach-3.1.0:
      Successfully uninstalled bleach-3.1.0
  Found exist

In [0]:
home_dir = '/content/gdrive/My Drive/Colab Notebooks/newlogic'
import sys
sys.path.append(home_dir)

In [0]:
model_path = home_dir + '/output/cnn-mnist-relu.pkl'

In [0]:
from cleverhans.attacks import SaliencyMapMethod
from cleverhans.dataset import MNIST
from cleverhans.loss import CrossEntropy
from cleverhans.utils import other_classes, set_log_level
from cleverhans.utils import pair_visual, grid_visual, AccuracyReport
from cleverhans.utils_tf import model_eval, model_argmax
from cleverhans.train import train
from cleverhans.utils_pytorch import convert_pytorch_model_to_tf
from cleverhans.model import CallableModelWrapper

In [0]:
import numpy as np
from six.moves import xrange
import tensorflow as tf
import torch
from torchvision import datasets, transforms

In [7]:
# Object used to keep track of (and return) key accuracies
report = AccuracyReport()

# Set TF random seed to improve reproducibility
tf.set_random_seed(1234)

# Create TF session and set as Keras backend session
sess = tf.Session()
print("Created TensorFlow session.")

Created TensorFlow session.


In [0]:
torch_model_orig = torch.load(model_path)
torch_model = lambda x: torch_model_orig(x)[0] # to standard format

In [0]:
tf_model_fn = convert_pytorch_model_to_tf(torch_model, out_dims=10)    
cleverhans_model = CallableModelWrapper(tf_model_fn, output_layer='logits')

In [0]:
jsma = SaliencyMapMethod(cleverhans_model, sess=sess)
jsma_params = {'theta': 1., 'gamma': 0.1,
                'clip_min': 0., 'clip_max': 1.,
                'y_target': None}

In [0]:
test_data = datasets.MNIST(home_dir + '/mnist', train=False, transform=transforms.ToTensor())

In [0]:
nb_classes = 10
source_samples = 10

In [0]:
test_loader = torch.utils.data.DataLoader(test_data, batch_size=source_samples)
x_test, y_test = next(iter(test_loader))

In [0]:
nchannels, img_rows, img_cols = x_test.shape[1:]

In [0]:
# Keep track of success (adversarial example classified in target)
results = np.zeros((nb_classes, source_samples), dtype='i')

# Rate of perturbed features for each test set example and target class
perturbations = np.zeros((nb_classes, source_samples), dtype='f')

# Initialize our array for grid visualization
grid_shape = (nb_classes, nb_classes, img_rows, img_cols, nchannels)
grid_viz_data = np.zeros(grid_shape, dtype='f')

In [38]:
for sample_ind in xrange(0, source_samples):
  print('--------------------------------------')
  print('Attacking input %i/%i' % (sample_ind + 1, source_samples))
  sample = x_test[sample_ind:(sample_ind + 1)]

  # We want to find an adversarial example for each possible target class
  # (i.e. all classes that differ from the label given in the dataset)
  current_class = int(np.argmax(y_test[sample_ind]))
  target_classes = other_classes(nb_classes, current_class)

  # For the grid visualization, keep original images along the diagonal
  grid_viz_data[current_class, current_class, :, :, :] = np.reshape(
      sample, (img_rows, img_cols, nchannels))

  # Loop over all target classes
  for target in target_classes:
    print('Generating adv. example for target class %i' % target)

    # This call runs the Jacobian-based saliency map approach
    one_hot_target = np.zeros((1, nb_classes), dtype=np.float32)
    one_hot_target[0, target] = 1
    jsma_params['y_target'] = tf.convert_to_tensor(one_hot_target)
    adv_x = jsma.generate(sample, **jsma_params)

    # Check if success was achieved
    res = int(model_argmax(sess, x, preds, adv_x) == target)

    # Compute number of modified features
    adv_x_reshape = adv_x.reshape(-1)
    test_in_reshape = x_test[sample_ind].reshape(-1)
    nb_changed = np.where(adv_x_reshape != test_in_reshape)[0].shape[0]
    percent_perturb = float(nb_changed) / adv_x.reshape(-1).shape[0]

    # Display the original and adversarial images side-by-side
    if viz_enabled:
      figure = pair_visual(
          np.reshape(sample, (img_rows, img_cols, nchannels)),
          np.reshape(adv_x, (img_rows, img_cols, nchannels)), figure)

    # Add our adversarial example to our grid data
    grid_viz_data[target, current_class, :, :, :] = np.reshape(
        adv_x, (img_rows, img_cols, nchannels))

    # Update the arrays for later analysis
    results[target, sample_ind] = res
    perturbations[target, sample_ind] = percent_perturb

--------------------------------------
Attacking input 1/10
Generating adv. example for target class 1


AttributeError: ignored

In [0]:
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)
for xs, ys in tqdm(test_loader):
    adv_preds = sess.run(adv_preds_op, feed_dict={x_op: xs})
    correct += (np.argmax(adv_preds, axis=1) == ys.cpu().detach().numpy()).sum()
    total += test_loader.batch_size
